### Load and Transform
This notebook represents the __Load__ and __Transform__ steps of data engineering. 

We are going to split the songs data into two relational tables so that the data is query ready. 

In [ ]:
%sql
-- SELECT DISTINCT Columns from raw_song_data
SELECT *
FROM raw_song_data
LIMIT 1;

In [ ]:
%sql
CREATE OR REPLACE TABLE
    artists (
        artist_id STRING,
        artist_lat DOUBLE,
        artist_long DOUBLE,
        artist_location STRING,
        artist_name STRING,
        processed_time TIMESTAMP -- Add timestamp
        );

INSERT INTO artists
    SELECT DISTINCT
        artist_id,
        artist_lat,
        artist_long,
        artist_location,
        artist_name,
        current_timestamp() -- Add timestamp
    FROM raw_song_data
    WHERE artist_id IS NOT NULL;

In [ ]:
%sql 
CREATE OR REPLACE TABLE
    songs (
        song_id STRING,
        title STRING,
        artist_id STRING,
        year INT,
        duration DOUBLE, -- Add duration category 
        length STRING,
        song_popularity DOUBLE, -- Change from hotnes
        loudness DOUBLE,
        processed_time TIMESTAMP -- Add timestamp
        );

INSERT INTO songs
    SELECT DISTINCT
        song_id,
        title,
        artist_id,
        NULLIF(year, 0) as year, -- Change to Null if 0
        duration,
        CASE
            WHEN duration < 120 THEN 'Short'                      -- Less than 2 minutes
            WHEN duration >= 120 AND duration < 300 THEN 'Medium' -- Less than 5 minutes
            WHEN duration >= 300 AND duration < 480 THEN 'Long'   -- Less than 8 minutes
            ELSE 'Extra Long'                                     -- Anything else
        END AS length,
        song_hotnes, -- Have to select original name
        loudness,
        current_timestamp() -- Add timestamp
    FROM raw_song_data
    WHERE song_id IS NOT NULL;